# Generalized Network Analysis 

The Network Analysis is part of the work entitled **Generalized correlation-based dynamical network analysis: a new high-performance approach for identifying allosteric communications in molecular dynamics trajectories**, by Marcelo C. R. Melo, Rafael C. Bernardi, Cesar de la Fuente-Nunez, and Zaida Luthey-Schulten. For more information see http://faculty.scs.illinois.edu/schulten/. 


In [1]:
# Load the python package
#import parmed as pmd
# Load the python package
import os
import dynetan
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *
from dynetan.contact import *


# Create the object that processes MD trajectories.
dnap = DNAproc()
dnap2 = DNAproc()

!pip install pytraj
import pytraj as pt
import sys, os
import numpy as np
#pip install mdtraj # Intall MDtraj separately
# import mdtraj as md
# from mdtraj.testing import get_fn
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
#import nglview as nv
from scipy.interpolate import griddata
#import moviepy.editor as mpy
# import seaborn as sb
# import barnaba
#import barnaba as bb
from statistics import mean, stdev
from pytraj import matrix
from matplotlib import colors
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('png')

/home/aakash/anaconda3/envs/pensa/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`


# Node Groups

In Network Analysis, each residue is represented by one or more "nodes", serving as proxies for groups of atoms form the original residue. This approach lowers computational cost and noise. 


# System info:
To get started, we need some basic information about the system:
- Work directory where we will find relevant files.
- Names of PSF and DCD files.
#- segid's for the regions that will be used for network analysis.
#- Name of the solvent residue(s) [default: TIP3]
#- Number of windows into which the trajectory will be split. [default: 4]
- Name of atom(s) that will represent node(s) in each residue.
- Names of atoms in node groups.

**You can can find a complete list of supported trajectory formats [here](https://www.mdanalysis.org/docs/documentation_pages/coordinates/init.html#supported-coordinate-formats):**


In [2]:
workDir = "/mnt/mainpool/storage/SHARED/Yog_Cas9/AAKASH_Simulations/final_simulations_AS/"

In [4]:
## -- Additional block for load and store smaller trajs, create without ion trajs  --- ##

ptop1 = os.path.join(workDir, "HF_0MM/SYS_nowat.prmtop")
ptop2 = os.path.join(workDir, "WT_0MM/SYS_nowat.prmtop")
# ptop3 = os.path.join(workDir, "atg_nowat_noion_newK674.prmtop")
f1 = os.path.join(workDir, "HF_0MM/traj_HF_0MM_R1-R2_4us_500_strided.dcd") ## Actually, skip100 = 200ps interval frames
f2 = os.path.join(workDir, "WT_0MM/traj_WT_0MM_R1-2_4us_strided.dcd")
# f3 = os.path.join(workDir, "5us/atgRNA_lag20ns_5us_tic1_skip10.dcd")

traj1 = pt.load(f1, top = ptop1, stride=1)

traj2 = pt.load(f2, top = ptop2, stride=1)

# traj3 = pt.load(f3, top = ptop3, stride=1)


# # traj1
ptop1_new = pt.strip(traj1.top, '@Mg+')
pt.save(os.path.join(workDir, "HF_0MM/HF_nowat_noion.prmtop"), ptop1_new)
traj1_new = pt.strip(traj1, '@Mg+')
pt.save(os.path.join(workDir, "HF_0MM/traj_HF_0MM_R1-R2_4us_500_strided_noion.dcd"), traj1_new)

# traj2
ptop2_new = pt.strip(traj2.top, '@Mg+')
pt.save(os.path.join(workDir, "WT_0MM/WT_nowat_noion.prmtop"), ptop2_new)
traj2_new = pt.strip(traj2, '@Mg+')
pt.save(os.path.join(workDir, "WT_0MM/traj_WT_0MM_R1-2_4us_strided_noion.dcd"), traj2_new)


In [3]:
## --  Block for traj and prmtop files  --- ##

ptop1 = os.path.join(workDir, "HF_0MM/HF_nowat_noion.prmtop")
ptop2 = os.path.join(workDir, "WT_0MM/WT_nowat_noion.prmtop")
f1 = os.path.join(workDir, "HF_0MM/traj_HF_0MM_R1-R2_4us_500_strided_noion.dcd") ## Actually, skip100 = 200ps interval frames
f2 = os.path.join(workDir, "WT_0MM/traj_WT_0MM_R1-2_4us_strided_noion.dcd")
   

In [4]:
# Path where input files will searched and results be written.
# Segment IDs for regions that will be studied.
segIDs = ["SYSTEM"]

# Number of windows created from full simulation.
#numWinds = 10
numWinds = 1 ## for cluster0

# Sampled frames per window
# numSampledFrames = 10

# Network Analysis will make one node per protein residue (in the alpha carbon)
# For other residues, the user must specify atom(s) that will represent a node.
customResNodes = {}
customResNodes["DC5"] = ["N1"] #
customResNodes["A5"] = ["N9"] #
customResNodes["DA5"] = ["N9"] #

customResNodes["A"] = ["P"]
customResNodes["C"] = ["P"]
customResNodes["G"] = ["P"]
customResNodes["U"] = ["P"]
customResNodes["DA"] = ["P"]
customResNodes["DC"] = ["P"]
customResNodes["DG"] = ["P"]
customResNodes["DT"] = ["P"]

customResNodes["G3"] = ["P"] #
customResNodes["DT3"] = ["P"] #
customResNodes["DG3"] = ["P"] #



# We also need to know the heavy atoms that compose each node group.

usrNodeGroups = {}

usrNodeGroups["DC5"] = {}
usrNodeGroups["A5"] = {}
usrNodeGroups["DA5"] = {}

usrNodeGroups["A"] = {}
usrNodeGroups["C"] = {}
usrNodeGroups["G"] = {}
usrNodeGroups["U"] = {}
usrNodeGroups["DA"] = {}
usrNodeGroups["DC"] = {}
usrNodeGroups["DG"] = {}
usrNodeGroups["DT"] = {}

usrNodeGroups["G3"] = {}
usrNodeGroups["DT3"] = {}
usrNodeGroups["DG3"] = {}

usrNodeGroups["DC5"]["N1"] = set("O5' C5' C4' O4' C1' N1 C3' C2' O3'".split())
usrNodeGroups["DA5"]["N9"] = set("O5' C5' C4' O4' C1' N9 C3' C2' O3'".split())
usrNodeGroups["A5"]["N9"] = set("O5' C5' C4' O4' C1' N9 C3' C2' O2' O3'".split())

usrNodeGroups["A"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O2' O3'".split())
usrNodeGroups["C"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O2' O3'".split())
usrNodeGroups["G"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O2' O3'".split())
usrNodeGroups["U"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O2' O3'".split())
usrNodeGroups["DA"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O3'".split())
usrNodeGroups["DC"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O3'".split())
usrNodeGroups["DG"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O3'".split())
usrNodeGroups["DT"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O3'".split())

usrNodeGroups["G3"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O2' O3'".split())
usrNodeGroups["DT3"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O3'".split())
usrNodeGroups["DG3"]["P"] = set("P OP1 OP2 O5' C5' C4' O4' C1' C3' C2' O3'".split())
#################################
### Extra configuration

# Cutoff for contact map (In Angstroms)
cutoffDist = 4.5

# Minimum contact persistance (In ratio of total trajectory frames)
contactPersistence = 0.50

#################################
### Load info to object

dnap.setNumWinds(numWinds)
# dnap.setNumSampledFrames(numSampledFrames)
dnap.setCutoffDist(cutoffDist)
dnap.setContactPersistence(contactPersistence)
#dnap.seth2oName(h2oName)
dnap.setSegIDs(segIDs)

dnap.setCustomResNodes(customResNodes)
dnap.setUsrNodeGroups(usrNodeGroups)

# Load the trajectory

Our Generalized Network Analysis leverages the MDAnalysis package to create a *universe* that contains all the trajectory and system information.

In [5]:
dnap.loadSystem(ptop2,f2)

In [6]:
# We can access the trajectory data directly.
dnap.getU().trajectory

<DCDReader /mnt/mainpool/storage/SHARED/Yog_Cas9/AAKASH_Simulations/final_simulations_AS/WT_0MM/traj_WT_0MM_R1-2_4us_strided_noion.dcd with 8000 frames of 26278 atoms>

### Checks segments and residue names

This is important to know if there are residues in the structure that we didn't know of, and need to be addresssed so that network analysis can create nodes in all selected residues.

In [7]:
dnap.checkSystem()

Residue verification:

---> SegID  SYSTEM : 34 unique residue types:
{'ILE', 'LYS', 'DG3', 'ARG', 'TRP', 'G', 'TYR', 'G3', 'A', 'GLY', 'ASN', 'C', 'DT3', 'DT', 'DA5', 'ASP', 'VAL', 'DC5', 'LEU', 'GLU', 'A5', 'ALA', 'PRO', 'PHE', 'U', 'DC', 'HIE', 'SER', 'MET', 'DG', 'DA', 'CYS', 'GLN', 'THR'}

---> 34 total selected residue types:
{'ILE', 'LYS', 'DG3', 'ARG', 'TRP', 'G', 'TYR', 'G3', 'A', 'GLY', 'ASN', 'C', 'DT3', 'DT', 'DA5', 'ASP', 'VAL', 'DC5', 'LEU', 'GLU', 'A5', 'ALA', 'PRO', 'PHE', 'U', 'DC', 'HIE', 'SER', 'MET', 'DG', 'DA', 'CYS', 'GLN', 'THR'}

---> 0 not-selected residue types in other segments:
set()

---> 34 total residue types:
{'ILE', 'LYS', 'DG3', 'ARG', 'TRP', 'G', 'TYR', 'G3', 'A', 'GLY', 'ASN', 'C', 'DT3', 'DT', 'DA5', 'ASP', 'VAL', 'DC5', 'LEU', 'GLU', 'A5', 'ALA', 'PRO', 'PHE', 'U', 'DC', 'HIE', 'SER', 'MET', 'DG', 'DA', 'CYS', 'GLN', 'THR'}

---> 1481 total residues were selected for network analysis.

Segments verification:

---> 1 selected segments:
['SYSTEM']

--

In [8]:
# More checks?
#
# You can add more checks here if your system has non-standard residues that need special attention.

### Prepare network representation of the system

Here we check that we know how to treat all types of residues in the final selection. Every residue will generate one or more nodes in the final network. Then we store the groups of atoms that define each node.

In [9]:
dnap.prepareNetwork()

Preparing nodes...


Nodes are ready for network analysis.


## Align the trajectory based on selected segments

We align the trajectory to its first frame using heavy atoms (non-hydrogen) from the selected segments. In the process, we also transfer the trajectory to the computer memory, so that future analysis and manipulations are completed faster.

In [10]:
# If your system is too large, you can turn off the "in memory" option, at a cost for performance.
dnap.alignTraj(inMemory=True)

  0%|          | 0/8000 [00:00<?, ?it/s]

### Select residues that are closer than 4.5A for more than 75% of simulation

<img align="right" src="./toolkit/figures/nodeContactDistances.png" alt="drawing" width="400px"/>

Creates an N-by-N matrix for all N nodes in the selected region, and automatically selected nodes (ions, solvent).

The following cell defines efficient functions to run the analysis and create a contact matrix. We leverage both MDAnalysis parallel contact detection tools, as well as accelerated Numba and Cython function. After creating the contact matrix, we remove any automatically selected nodes that have insuficient persistance, and filter the contacts by (optionally) removing contacts between nodes in consecutive residues in a protein or nucleic chain.

In the example to the right, the distances between atoms indicates that in that frame of the simulation, the node representing the water molecule would make a contact with the phosphate node (representing the ribose ring and phosphate), but not with the nitrogen node (representing the nucleic base).



**Attention** For every time you start this Jupyter Notebook, the first time you execute this function may take significanlty longer (several seconds) to start. This is because we use *Cython* and *Numba* to compile functions "on-demand", and a new compilation may be necessary after the notebook is re-started.

In [11]:
# To speed-up the contact matrix calculation, a larger stride can be selected, at a cost for precision.
dnap.findContacts(stride=10)

We found 0 nodes with no contacts.
We found 7997 contacting pairs out of 1095940 total pairs of nodes.
(That's 0.7%, by the way)


In [10]:
# dir(dnap.findContacts)

### Removing contacts between nodes in the same residue.

The following function guarantees that there will be no "self-contacts" (contacts between a node and itself), and gives you the opportunity to remove contacts between nodes in consecutive residues (such as sequential amino acids in the same chain, removing back-bone interactions). 

The function also removes nodes that are never in contact with any other node in the system (such as the ends of flexible chains, or residues in flexible loops). This will automatically update the MDanalysis universe and related network informatio, such as number of nodes and atom-to-node mappings.

In [13]:
# dnap.filterContacts(notSameRes=True, notConsecutiveRes=False, removeIsolatedNodes=True)

In [14]:
# import sys
# print(sys.getrecursionlimit())
# sys.setrecursionlimit(4500)

## Calculate Generalized Correlation with Python/Numba

In [12]:
# We can calculate generalized correlaions in parallel using Python's multiprocessing package.
dnap.calcCor(ncores=36)

Calculating correlations...

Using window length of 8000 simulation steps.
- > Using multi-core implementation with 36 threads.


## Calculate cartesian distances between all nodes in the selected system.

Here, we will calculate the **shortest** distance between atoms in all pairs of nodes. It is similar to the contact matrix calculation, but we check all distances and keep the shortest one to use in our analysis.

In [13]:
# We can leverage MDanalysis parallelization options with backend="serial" or backend="openmp".
# For very small systems, the serial can be faster!
# Sampled frames per window
numSampledFrames = 50
dnap.setNumSampledFrames(numSampledFrames)
dnap.calcCartesian(backend="openmp")

Sampling a total of 50 frames from 1 windows (50 per window)...


# Network Calculations
Create a graph from our correlation matrix. Different properties are calculated:

*Density* maesures how connected the graph is compared to how connected it *could* be. It is the ratio between edges in the graph over all possible edges between all pairs of nodes.

*Transitivity* maesures the triadic closure, comparing present triangles to possible triangles. In a triangle, if A is connected to B, and B connected to C, then A is connected to C.

*Degree* measures the number of connections a node has.

(Reference)[1]

[1]:https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python#advanced-networkx-community-detection-with-modularity

In [14]:
dnap.calcGraphInfo()

In [15]:
# Basic information of the network as interpreted as a graph.
with open(os.path.join(workDir, "WT_0MM/Network_Analysis/WT0MM_network_info.txt"), 'w') as f:
    print( nx.info(dnap.nxGraphs[0]), file = f)
    print( 'Diameter of the network:', nx.diameter(dnap.nxGraphs[0]), file = f)
    # Both density and transitivity are scaled from 0 to 1
    for win in range(dnap.numWinds):
#         print("----- Window {} -----".format(win),)
        print("Density:", round( nx.density(dnap.nxGraphs[win]), 4), file = f )
        print("Transitivity:", round( nx.transitivity(dnap.nxGraphs[win]), 4),"\n", file =f )
        
# print( nx.info(dnap.nxGraphs[0]))

In [16]:
from operator import itemgetter

# We can check the nodes that have the most connections in each window.
# with open('crRNA_network_info.txt', 'a') as f:
with open(os.path.join(workDir, "WT_0MM/Network_Analysis/WT0MM_sorted_nodes_by_degree.txt"), 'a') as f:
    
    for win in range(dnap.numWinds):
        print("----- Window {} -----".format(win))

        sorted_degree = sorted(dnap.getDegreeDict(win).items(), key=itemgetter(1), reverse=True)

#         print("Top 20 nodes by degree: [node --> degree : selection]", file = f)
#         for n,d in sorted_degree[:20]:
        for n,d in sorted_degree[:]:
#             print("{0:>4} --> {1:>2} : {2}".format(n, d, getSelFromNode(n, dnap.nodesAtmSel)))
            print("{0:>4} --> {1:>2} : {2}".format(n, d, getSelFromNode(n, dnap.nodesAtmSel)), file = f)

        print()

----- Window 0 -----



In [69]:
# dir(dnap)

## Calculate optimal paths
We choose the Floyd Warshall algorithm[1]. This uses the **correlations as weights** to calculate network distances and shortest distances.

[1]:https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.dense.floyd_warshall.html?highlight=warshall#networkx.algorithms.shortest_paths.dense.floyd_warshall

In [17]:
dnap.calcOptPaths(ncores=12)

## Calculate betweenness

We calculate both betweenness centrality[1] for edges and eigenvector centrality[2] for nodes.

[1]:https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html?highlight=betweenness#networkx.algorithms.centrality.edge_betweenness_centrality
[2]:https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.eigenvector_centrality.html

In [18]:
dnap.calcBetween(ncores=12)

In [19]:
from itertools import islice

# Pairs of nodes with highest Betweeness values, compared to their correlation values (in Window 0)
with open(os.path.join(workDir, "WT_0MM/Network_Analysis/WT0MM_sorted_nodes_by_betweenness.txt"), 'a') as f:
    
#     for k,v in islice(dnap.btws[0].items(),10):
    for k,v in islice(dnap.btws[0].items(),None):
        print("Nodes {} have betweenes {} and correlation {}.".format(k, 
                                                                      round(v,3), 
                                                                      round(dnap.corrMatAll[0, k[0], k[1]], 3) ), file = f )

Turn to node centrality instead of edge centrality:

In [20]:
dnap.calcEigenCentral()

### Calculate communities



Using **Louvain heuristices** is feasible. 
This method also maximizes the modularity of the network.

http://iopscience.iop.org/article/10.1088/1742-5468/2008/10/P10008/meta

In [21]:
dnap.calcCommunities()

In [22]:
# Sort communities based on number of nodes
for comIndx in dnap.nodesComm[0]["commOrderSize"]:
    print("Modularity Class {0:>2}: {1:>3} nodes.".format(comIndx, len(dnap.nodesComm[0]["commNodes"][comIndx])))

Modularity Class  0: 153 nodes.
Modularity Class  4: 149 nodes.
Modularity Class  6: 139 nodes.
Modularity Class  7: 132 nodes.
Modularity Class  9: 126 nodes.
Modularity Class 11: 122 nodes.
Modularity Class 10: 119 nodes.
Modularity Class  8: 116 nodes.
Modularity Class 12:  88 nodes.
Modularity Class 13:  86 nodes.
Modularity Class  2:  72 nodes.
Modularity Class  3:  69 nodes.
Modularity Class  5:  59 nodes.
Modularity Class  1:  51 nodes.


In [23]:
# Sort communities based on the node with highest eigenvector centrality
with open(os.path.join(workDir, 'WT_0MM/Network_Analysis/WT0MM_community_EigenCentrality.txt'), 'a') as f:
    for comIndx in dnap.nodesComm[0]["commOrderEigenCentr"]:
        print("Modularity Class {0} ({1} nodes) Sorted by Eigenvector Centrality:".format(
                                                 comIndx, len(dnap.nodesComm[0]["commNodes"][comIndx])),  file = f)
#         for node in dnap.nodesComm[0]["commNodes"][comIndx][:5]:
        for node in dnap.nodesComm[0]["commNodes"][comIndx][:]:
            print("Name: {0:>4} | Degree: {1:>2} | Eigenvector Centrality: {2}".format(
                node, dnap.nxGraphs[win].nodes[node]['degree'], dnap.nxGraphs[win].nodes[node]['eigenvector']), file = f)
            print()

# Process Interface Residues

We now find all nodes that are close to both selections chosen by the user. That may include amino acids in the interface, as well as ligands, waters and ions.

In [24]:
dnap.interfaceAnalysis(selAstr="resid 1:1153", selBstr="resid 1154:1204")

0 pairs of nodes connecting the two selections.
0 unique nodes in interface node pairs.


## Save The Data

In [25]:
pathToData = "/mnt/mainpool/storage/SHARED/Yog_Cas9/AAKASH_Simulations/final_simulations_AS/WT_0MM/Network_Analysis/"

fileNameRoot = "WT0MM_network"

fullPathRoot = os.path.join(pathToData, fileNameRoot)

dnap.saveData(fullPathRoot)

## Save reduced trajectory for visualization

In [26]:
# This function will save a reduced DCD trajectory with the heavy atoms used for network analysis
# A smaller trajectory can be created by choosing a "stride" that sub-samples the original trajectory.
# This function will also produce a PDB file so that information on atoms and residues can be loaded to
#    visualization software such as VMD.

dcdstride = 1

print("We will save {} heavy atoms and {} frames.".format(dnap.workU.atoms.n_atoms, 
                                                          len(dnap.workU.trajectory[::dcdstride]) ))

We will save 26278 heavy atoms and 8000 frames.


In [27]:
dnap.saveReducedTraj(fullPathRoot, stride = dcdstride)

/home/aakash/anaconda3/envs/pensa/lib/python3.7/site-packages/MDAnalysis/coordinates/DCD.py:430: UserWarning: No dimensions set for current frame, zeroed unitcell will be written
  warnings.warn(wmsg)
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this lim

MDAnalysis may print warnings regarding missing data fields, such as altLocs, icodes, occupancies, or tempfactor, which provide information commonly found in PDB files.
The warnings are for your information and in the context of this tutorial they are expected and do not indicate a problem.

# Analysis

The we have finished processing the trajectory and storing all related data. We can now move on to analysis of the network properties calculated here.

**All analysis code was placed in a second tutorial notebook for clarity.**


# ---- The End ----